In [30]:
import pandas as pd

df = pd.read_csv('UL-Standards-List.csv')
# drop rows with no description
df = df.dropna(subset=['Description'])
len(df)
# df.head()


2123

In [31]:
# Description to Title Case since there are some descriptions that are all caps
df['Description'] = df['Description'].str.title()





In [32]:

df.iloc[144]['Description']

'Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method) / Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method)'

In [33]:
# Removing duplicates in the description separated by ' / '
def remove_duplicates(string):
    words = string.split(' / ')
    result = []
    for word in words:
        if word not in result:
            result.append(word)
    return ' / '.join(result)  # nothing to join if there are no duplicates

df['Description'] = df['Description'].apply(remove_duplicates)
df.iloc[570]['Description']

'Standard For Category 3 Film-Forming Fluoroprotein (Fffp) Foam Liquid Concentrates'

In [34]:
# Look for non-English descriptions
def isEnglish(s):
    return s.isascii()

df['English?'] = df['Description'].apply(isEnglish)


In [35]:
df['Category'] = ''
df['Topic'] = ''

In [36]:
# Remove non-English descriptions
for idx, row in df.iterrows():
    if row['English?'] == False:
        split_description = row['Description'].split(' / ', maxsplit=1)
        if len(split_description) > 1:
            df.at[idx, 'Topic'] = split_description[0]
        else:
            df.at[idx, 'Topic'] = row['Description']
    else:
        df.at[idx, 'Topic'] = row['Description']



In [37]:
import re

# Split at ';' or ':' or '-'
def split_topic_punc(string):
    split_string = re.split(';|:| - ', row['Topic'], maxsplit=1)
    if len(split_string) > 1:
        return split_string
    else:
        return string

for idx, row in df.iterrows():
    df.at[idx, 'Category'] = split_topic_punc(row['Topic'])[0]
    df.at[idx, 'Topic'] = split_topic_punc(row['Topic'])[1]


In [39]:
df['Category'][1505]

'Ieee/Ul Wireless Diabetes Device Security'

In [ ]:

def find_words_in_description(string, words, stop_chars):
    if stop_chars:
        # Find the index of the first occurrence of any of the stop characters
        stop_index = re.search(stop_chars, string)
        if stop_index:
            # Only search for words in the part of the string before the stop character
            string = string[:stop_index.start()]
    start_index = 0
    found_word = ''
    for word in words:
        i = string.find(word, start_index)
        if i != -1:
            start_index = i + len(word)
            found_word = found_word + ' ' + word
        else:
            start_index = start_index
    return (start_index, found_word)

words = ['Standard', 'Method', 'Test', 'Outline', 'Guide', 'Guidance', 'Criteria', 'Procedure', 'Rule', 'Requirement']
stop_chars = '[,;:]'

for idx, row in df.iterrows():
    i = find_words_in_description(row['Topic'], words, stop_chars=stop_chars)[0]
    final_word = find_words_in_description(row['Topic'], words, stop_chars=stop_chars)[1]
    print(i, final_word)
    if final_word != '':
        df.at[idx, 'Category'] = final_word
        split_topic_desc = re.split(' For | Of ', row['Topic'][i:], maxsplit=1)
        # join the word in description before the split if not in the list of words

        print(split_topic_desc)
        if len(split_topic_desc) > 1:
            lost_words = [word for word in row['Topic'][:i] if word not in words]
            new_string = ''.join(lost_words)
            df.at[idx, 'Category'] = new_string
            df.at[idx, 'Topic'] = split_topic_desc[1]

    else:
        pass

In [32]:
for index, row in df.iterrows():
    i = row['Description'].find('Standard')

    if i != -1:
        df.at[index, 'Category'] = 'Standard'
        split_description = row['Description'][i+8:].split(' For ', maxsplit=1)
        if len(split_description) > 1:
            print(split_description)
            df.at[index, 'Topic'] = split_description[1]
    else:
        df.at[index, 'Description'] = row['Description']
df

In [4]:
# from top2vec import Top2Vec
# model = Top2Vec(documents=df['Description'].tolist())



KeyboardInterrupt

